In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2

from camera import Camera
import structure
import processor
import features

In [3]:
img1 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0289.JPG')
img2 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0288.JPG')
img3 = cv2.imread('../eigenerAnsatz/bildverband2/DJI_0287.JPG')
print("Bilder geladen")

Bilder geladen


In [4]:
#pts1, pts2 = features.find_correspondence_points(img1, img2)
#print("Punkte gefunden")

Punkte gefunden


In [19]:
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(
    cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None)
kp2, des2 = sift.detectAndCompute(
    cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None)
kp3, des3 = sift.detectAndCompute(
    cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY), None)

def matchPoints(kp1,des1,kp2,des2):
    # Find point matches
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=100)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Apply Lowe's SIFT matching ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)

    src_pts = np.asarray([kp1[m.queryIdx].pt for m in good])
    dst_pts = np.asarray([kp2[m.trainIdx].pt for m in good])

    # Constrain matches to fit homography
    retval, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 100.0)
    mask = mask.ravel()

    # We select only inlier points
    pts1 = src_pts[mask == 1]
    pts2 = dst_pts[mask == 1]
    return pts1.T, pts2.T


pts1, pts2a = matchPoints(kp1, des1, kp2, des2)

pts2b, pts3 = matchPoints(kp1, des1, kp2, des2)


In [5]:
points1 = processor.cart2hom(pts1)
points2a = processor.cart2hom(pts2a)

points2b = processor.cart2hom(pts2a)
points3 = processor.cart2hom(pts3)


In [32]:
points1

array([[5.82781887e+00, 7.61792803e+00, 8.68769646e+00, ...,
        3.98936694e+03, 3.99002026e+03, 3.99109863e+03],
       [6.81190308e+02, 9.98320007e+02, 7.18246582e+02, ...,
        1.41994214e+03, 2.03552795e+03, 1.17817090e+03],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [6]:
fig, ax = plt.subplots(1, 3)
ax[0].autoscale_view('tight')
ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
ax[0].plot(points1[0], points1[1], 'r.')
ax[1].autoscale_view('tight')
ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[1].plot(points2[0], points2[1], 'r.')
ax[2].autoscale_view('tight')
ax[2].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
ax[2].plot(points2[0], points2[1], 'r.')
fig.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [7]:
height, width, ch = img1.shape
intrinsic = np.array([  # for dino
    [3030.65, 0, width / 2-6],
    [0, 3030.65, height / 2+17],
    [0, 0, 1]])
intrinsic

array([[3.03065e+03, 0.00000e+00, 1.99400e+03],
       [0.00000e+00, 3.03065e+03, 1.51700e+03],
       [0.00000e+00, 0.00000e+00, 1.00000e+00]])

In [8]:
# Calculate essential matrix with 2d points.
# Result will be up to a scale
# First, normalize points
points1n = np.dot(np.linalg.inv(intrinsic), points1)
points2n = np.dot(np.linalg.inv(intrinsic), points2)
E = structure.compute_essential_normalized(points1n, points2n)
print('Computed essential matrix:', (-E / E[0][1]))

Computed essential matrix: [[-0.03157542 -1.         -0.23135702]
 [ 0.95870465 -0.01264242 -0.00189078]
 [ 0.25888847  0.20644202  0.04803392]]


Comparison to openCV

In [31]:
E2, mask = cv2.findEssentialMat(points1n[:2].T, points2n[:2].T)
# E2 = np.linalg.inv(E2)
-E2 / E2[0][1]

array([[-0.02138276, -1.        , -0.31633499],
       [ 0.98961303,  0.04504575, -0.21970331],
       [ 0.26023036,  0.02401117, -0.05386352]])

In [ ]:

retval, R, t, mask = cv2.recoverPose(
    E2, pts1.T, pts2.T, intrinsic)
E2,R,t

In [10]:

# Given we are at camera 1, calculate the parameters for camera 2
# Using the essential matrix returns 4 possible camera paramters
P1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
P2s = structure.compute_P_from_essential(E)
P2s

[array([[-0.98023525, -0.04897094,  0.19167862,  0.19881356],
        [-0.04587688, -0.88620151, -0.46102299, -0.24938911],
        [ 0.19244261, -0.4607046 ,  0.86643933,  0.94777542]]),
 array([[-0.98023525, -0.04897094,  0.19167862, -0.19881356],
        [-0.04587688, -0.88620151, -0.46102299,  0.24938911],
        [ 0.19244261, -0.4607046 ,  0.86643933, -0.94777542]]),
 array([[ 0.97981755, -0.04064291,  0.19571847,  0.19881356],
        [ 0.04640084,  0.99861194, -0.02492283, -0.24938911],
        [-0.19443387,  0.03350133,  0.98034337,  0.94777542]]),
 array([[ 0.97981755, -0.04064291,  0.19571847, -0.19881356],
        [ 0.04640084,  0.99861194, -0.02492283,  0.24938911],
        [-0.19443387,  0.03350133,  0.98034337, -0.94777542]])]

In [11]:
ind = -1
for i, P2 in enumerate(P2s):
    # Find the correct camera parameters
    d1 = structure.reconstruct_one_point(
        points1n[:, 0], points2n[:, 0], P1, P2)

    # Convert P2 from camera view to world view
    P2_homogenous = np.linalg.inv(np.vstack([P2, [0, 0, 0, 1]]))
    d2 = P2_homogenous[:3, :4] @ d1

    if d1[2] > 0 and d2[2] > 0:
        ind = i

P2 = np.linalg.inv(np.vstack([P2s[ind], [0, 0, 0, 1]]))[:3, :4]
print(P2)

[[ 0.97981755  0.04640084 -0.19443387 -0.00105049]
 [-0.04064291  0.99861194  0.03350133 -0.22537157]
 [ 0.19571847 -0.02492283  0.98034337  0.97427232]]


Comparison to openCV

In [12]:
retval, R, t, mask = cv2.recoverPose(E2,pts1.T,pts2.T,intrinsic)
P2inv = np.hstack([R, t])
np.linalg.inv(np.vstack([P2inv,[0,0,0,1]]))

array([[ 0.97884388, -0.03770837,  0.20110382, -0.21131096],
       [ 0.03815494,  0.99927046,  0.0016565 ,  0.18434596],
       [-0.20101957,  0.00605165,  0.97956853, -0.9598772 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [17]:
#tripoints3d = structure.reconstruct_points(points1n, points2n, P1, P2)
tripoints3d = structure.linear_triangulation(points1n, points2n, P1, P2)
tripoints3d

array([[-1.00553646, -1.01953347, -1.00409616, ...,  0.87509442,
         0.89586751,  0.86850577],
       [-0.42699544, -0.27009344, -0.40814642, ..., -0.03954603,
         0.23656618, -0.1444281 ],
       [ 1.53287928,  1.5548741 ,  1.53279683, ...,  1.33156662,
         1.3647532 ,  1.31957639],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ]])

In [14]:
tri = cv2.triangulatePoints(P1,P2,pts1,pts2)
#tripoints3d = processor.hom2cart(tri)
tripoints3d = cv2.convertPointsFromHomogeneous(tri.T).T


In [18]:
%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'b.')
ax.plot([0], [0], [0], 'r.')
ax.plot(-P2[0, 3], -P2[1, 3], -P2[2, 3], 'r.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
